# Reliability Test
Computes concentration metrics and the **Behavioral Gap** ratio between Anchors and Noise.

In [5]:
import pandas as pd
from pathlib import Path

In [6]:
DATA_DIR = Path("../data/processed")
input_path = DATA_DIR / "hourly_validation_metrics.csv"
output_path = DATA_DIR / "behavioral_concentration_metrics.csv"

if not input_path.exists():
    raise FileNotFoundError("hourly_validation_metrics.csv not found.")

print("Starting Concentration Metrics Validation...")
hourly_df = pd.read_csv(input_path)

Starting Concentration Metrics Validation...


In [7]:
results = []
for segment in hourly_df['final_status'].unique():
    segment_data = hourly_df[hourly_df['final_status'] == segment]
    sorted_by_val = segment_data.sort_values(by='pct_of_daily_rides', ascending=False)
    top2_concentration = sorted_by_val.head(2)['pct_of_daily_rides'].sum()
    std_dev = segment_data['pct_of_daily_rides'].std()
    results.append({
        "Segment": segment,
        "Top 2 Hour Concentration (%)": round(top2_concentration, 2),
        "Hourly Std Deviation": round(std_dev, 2)
    })

metrics_df = pd.DataFrame(results)
metrics_df.to_csv(output_path, index=False)
metrics_df

,Segment,Top 2 Hour Concentration (%),Hourly Std Deviation
0,Confirmed Behavioral Anchor,30.84,4.43
1,Inconsistent / Noise,18.75,2.95


In [8]:
anchor_stats = metrics_df[metrics_df['Segment'] == 'Confirmed Behavioral Anchor']
noise_stats = metrics_df[metrics_df['Segment'] == 'Inconsistent / Noise']

if not anchor_stats.empty and not noise_stats.empty:
    a_conc = anchor_stats['Top 2 Hour Concentration (%)'].values[0]
    n_conc = noise_stats['Top 2 Hour Concentration (%)'].values[0]
    ratio = a_conc / n_conc
    print(f"FINAL BEHAVIORAL GAP: {ratio:.2f}x")
    if ratio >= 1.5:
        print("DECISION: ELITE SEPARATION")
    elif ratio >= 1.25:
        print("DECISION: CLEAR SEPARATION")
    else:
        print("DECISION: WEAK SEPARATION")

FINAL BEHAVIORAL GAP: 1.64x
DECISION: ELITE SEPARATION
